This notebook is part of the `nbsphinx` documentation: https://nbsphinx.readthedocs.io/.

# Prolog and Epilog

When including notebooks in your Sphinx documentation, you can choose to add some generic content before and after each notebook.
This can be done with the configuration values `nbsphinx_prolog` and `nbsphinx_epilog` in the file `conf.py`.

The prolog and epilog strings can hold arbitrary [reST](https://www.sphinx-doc.org/rest.html) markup.
Particularly, the [only](https://www.sphinx-doc.org/en/master/usage/restructuredtext/directives.html#directive-only) and [raw](https://docutils.readthedocs.io/en/sphinx-docs/ref/rst/directives.html#raw-data-pass-through) directives can be used to have different content for HTML and LaTeX output.

Those strings are also processed by the [Jinja2](http://jinja.pocoo.org/) templating engine.
This means you can run Python-like code within those strings.
You have access to the current [Sphinx build environment](https://www.sphinx-doc.org/en/master/extdev/envapi.html) via the variable `env`.
Most notably, you can get the file name of the current notebook with

    {{ env.doc2path(env.docname, base=None) }}

Have a look at the [Jinja2 template documentation](http://jinja.pocoo.org/docs/latest/templates/) for more information.

<div class="alert alert-warning">

Warning

If you use invalid syntax, you might get an error like this:

    jinja2.exceptions.TemplateSyntaxError: expected token ':', got '}'

This is especially prone to happen when using raw LaTeX, with its abundance of braces.
To avoid clashing braces you can try to insert additional spaces or LaTeX macros that don't have a visible effect, like e.g. `\strut{}`.
For example, you can avoid three consecutive opening braces with something like that:

    \texttt{\strut{}{{ env.doc2path(env.docname, base=None) }}}

NB: The three consecutive closing braces in this example are not problematic.
    
An alternative work-around would be to surround LaTeX braces with Jinja braces like this:
    
    {{ '{' }}
    
The string within will not be touched by Jinja.

Another special Jinja syntax is `{%`, which is also often used in fancy TeX/LaTeX code.
A work-around for this situation would be to use
    
    {{ '{%' }}

</div>

## Examples

You can include a simple static string, using [reST](https://www.sphinx-doc.org/rest.html) markup if you like:

```python
nbsphinx_epilog = """
----

Generated by nbsphinx_ from a Jupyter_ notebook.

.. _nbsphinx: https://nbsphinx.readthedocs.io/
.. _Jupyter: https://jupyter.org/
"""
```

Using some additional Jinja2 markup and the information from the `env` variable, you can create URLs that point to the current notebook file, but located on some other server:

```python
nbsphinx_prolog = """
Go there: https://example.org/notebooks/{{ env.doc2path(env.docname, base=None) }}

----
"""
```

You can also use separate content for HTML and LaTeX output, e.g.:

```python
nbsphinx_prolog = r"""
{% set docname = env.doc2path(env.docname, base=None) %}

.. only:: html

    Go there: https://example.org/notebooks/{{ docname }}

.. raw:: latex

    \nbsphinxstartnotebook{The following section was created from
    \texttt{\strut{}{{ docname }}}:}
"""

nbsphinx_epilog = r"""
.. raw:: latex

    \nbsphinxstopnotebook{\hfill End of notebook.}
"""
```

Note the use of the `\nbsphinxstartnotebook` and `\nbsphinxstopnotebook` commands.
Those make sure there is not too much space between the "prolog" and the beginning of the notebook and, respectively, between the end of the notebook and the "epilog".
They also avoid page breaks, in order for the "prolog"/"epilog" not to end up on the page before/after the notebook.

For a more involved example for different HTML and LaTeX versions, see the file [conf.py](conf.py) of the `nbsphinx` documentation.